In [26]:
import os
import pandas as pd
import requests
import getpass
from bs4 import BeautifulSoup
import json
from pandas.io.json import json_normalize

In [2]:
key = getpass.getpass()

········


In [3]:
def summoner_name(key, name):
    name = name.replace(' ', '%20')
    request = requests.get(f'https://br1.api.riotgames.com/tft/summoner/v1/summoners/by-name/{name}?api_key={key}')
    soup = BeautifulSoup(request.content)
    return eval(soup.text)

In [4]:
 teste = summoner_name(key, 'Es mir egal')

In [5]:
pd.DataFrame([teste])

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,4GfUXIlMw5B040aVRVjxDlHVKkhiy4DP8T_HjdeuxG-QoV8,TYPXovIvidbWnodF3_pVrS-Uz7BKg-1x_7LWyVTvrLFV7-U,K3mbUvb-2PPBom5fdP5FBdn-2CUsQqy4t2wh-dmw-e55tv...,Es mir egal,4498,1587845371000,84


In [15]:
def match_list(key, puuid, n = 30):
    request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count={n}&api_key={key}')
    soup = BeautifulSoup(request.content)
    return eval(soup.text)

In [16]:
match_list(key, teste['puuid'])

['BR1_1926730533',
 'BR1_1926675398',
 'BR1_1926670564',
 'BR1_1926614700',
 'BR1_1926538340',
 'BR1_1926140313',
 'BR1_1926073659',
 'BR1_1926067196',
 'BR1_1926063272',
 'BR1_1925978166',
 'BR1_1925750238',
 'BR1_1925712925',
 'BR1_1925627632',
 'BR1_1925583474',
 'BR1_1925376688',
 'BR1_1925432251',
 'BR1_1925266933',
 'BR1_1925232809',
 'BR1_1925207099',
 'BR1_1919021209',
 'BR1_1918904380',
 'BR1_1918819128',
 'BR1_1918810872',
 'BR1_1918745440',
 'BR1_1918760191',
 'BR1_1918703027',
 'BR1_1918570224',
 'BR1_1918505797',
 'BR1_1918435816',
 'BR1_1918461173']

In [78]:
def match_info(key, matchid):
    '''Return a dataframe with some infos from the match (matchid, datetime, game lenght(s), game variation,
    game version, number of the set).
    
    key = riot API-key
    matchid = The id from a specific match of team fight tatics(TFT)'''
    
    request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/{matchid}?api_key={key}')
    soup = BeautifulSoup(request.content)
    df = pd.DataFrame(pd.json_normalize(eval(soup.text)))
    df2 = df[['metadata.match_id', 'info.game_datetime', 'info.game_length', 'info.game_variation',
       'info.game_version', 'info.tft_set_number']]
    df2.columns = [i.replace('metadata.', '').replace('info.', '') for i in df2.columns]
    
    return df2

In [79]:
match_info(key, 'BR1_1926730533')

,match_id,game_datetime,game_length,game_variation,game_version,tft_set_number
0,BR1_1926730533,1587870567039,2199.608887,TFT3_GameVariation_FreeNeekos,Version 10.8.317.8137 (Apr 17 2020/16:09:11) [...,3


In [74]:
def match_participants(key, matchid):
    '''Return a list with the puuid from the 8 participants of the match'''
    
    request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/{matchid}?api_key={key}')
    soup = BeautifulSoup(request.content)
    df = pd.DataFrame(pd.json_normalize(eval(soup.text)))
    return df.iloc[0,2]

In [75]:
match_participants(key, 'BR1_1926730533')

['qYjGRXS83Z7LIC4c40Hk27k2ee6SrYKWhGOwdRbz2IjXf42FEWv-dvIcisNE9lEFbPwVzkCCJ34KRQ',
 'ZOp977RUzO2rc4XXpC5LDmSW9FVPinsos54x01UvxLH98PpP3NYXmClEjBL7z1YWn1NsOFa9JXtPIg',
 'BsmmhSCiK5vK67zbbMhAz2edViDJ0dvNlTuQOZQNcbu13xPKgDvxptOsdCmg3nfAfFl4wBy2YqtVAQ',
 'Nd0RYeDsIefGnmlGA45x6zj-thg349ozYFcEobPFfROdqdBCF9Gs3SeWzIgqagknLSiDqgxEqqNUiw',
 'VZV01cG8R-A-yZ5spGtEhMTOfsdVBZhf-4EDIkcEhgjb3M8botGBmNEa64FxmNGUxiHyydTByqVWyA',
 'K3mbUvb-2PPBom5fdP5FBdn-2CUsQqy4t2wh-dmw-e55tv0nVirYpqo-AISeib2MLctqZ63zorohUg',
 'yWg9adnTaYHLhXpyOgkDilt9Oyv80_n4-RlL19ZpQtnXpOWgHRcAsXcynltTP4w9hi0xhQ9VbK14Cg',
 'OpAqJOjE1o8Y1l3cb50mh1iaUnDONPCzEZUm8fobWVot-6wBa8ltTlG6joXoyEYSqJ8ladkieMBW5g']